In [1]:
import numpy as np
import pandas as pd
from glob import glob
import gzip
import shutil
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import math

In [2]:
instruments = ['USD000000TOD', 'USD000UTSTOM', 'EUR_RUB__TOD', 'EUR_RUB__TOM', 'EURUSD000TOM', 'EURUSD000TOD']

instruments_info = {'USD000000TOD': {'SCHEDULE': 174500000000, 'PRICE_STEP': 0.0025},
                    'USD000UTSTOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025},
                    'EUR_RUB__TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.0025},
                    'EUR_RUB__TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025},
                    'EURUSD000TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.00001},
                    'EURUSD000TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.00001}}

In [3]:
class Spectrum:
    def __init__(self, price_step):
        self.price_step = price_step
        self.buy_bins = []
        self.all_buy_bins = []
        self.all_buy_volumes = []
        self.all_sell_bins = []
        self.all_sell_volumes = []
        self.sell_bins = []
        self.best_sell = -1
        self.best_buy = -1
        self.buy_volumes = [0 for i in range(50)]
        self.sell_volumes = [0 for i in range(50)]
        self.spectrum_sell_bins = []
        self.spectrum_buy_bins = []
        self.spectrum_sell = [0 for i in range(10)]
        self.spectrum_buy = [0 for i in range(10)]
                

    def insert_to_buy(self, price, volume):
        max_price = self.all_buy_bins[0]
        # new one is greater than the currently recorded maximum
        if price > max_price:
            dist = math.ceil((price - max_price) / self.price_step)
            self.all_buy_bins = [i for i in np.arange(price, max_price, -self.price_step)] + self.all_buy_bins
            self.all_buy_volumes = [0 for i in range(len(self.all_buy_bins) - len(self.all_buy_volumes))] + self.all_buy_volumes
            self.all_buy_volumes[0] += volume
            return 0
        else:
            idx = math.ceil((max_price - price) / self.price_step)
            if idx < len(self.all_buy_bins):
                self.all_buy_volumes[idx] += volume
                return idx
            else:
                dist = idx - len(self.all_buy_bins) + 1
                self.all_buy_bins = self.all_buy_bins + [i for i in np.arange(self.all_buy_bins[-1] - self.price_step, price - 1, -self.price_step)]
                self.all_buy_volumes = self.all_buy_volumes + [0 for i in range(len(self.all_buy_bins) - len(self.all_buy_volumes))]
                self.all_buy_volumes[idx] += volume
                return idx

    def insert_to_sell(self, price, volume):
        min_price = self.all_sell_bins[0]
        # new one is less than the currently recorded minimum
        if price < min_price:
            dist = math.ceil((min_price - price) / self.price_step)
            self.all_sell_bins = [i for i in np.arange(price, min_price, self.price_step)] + self.all_sell_bins
            self.all_sell_volumes = [0 for i in range(len(self.all_sell_bins) - len(self.all_sell_volumes))] + self.all_sell_volumes
            self.all_sell_volumes[0] += volume
            return 0
        else:
            idx = math.ceil((price - min_price) / self.price_step)
            if idx < len(self.all_sell_bins):
                self.all_sell_volumes[idx] += volume
                return idx
            else:
                dist = idx - len(self.all_sell_bins) + 1
                self.all_sell_bins = self.all_sell_bins + [i for i in np.arange(self.all_sell_bins[-1] + self.price_step, price + 1, self.price_step)]
                self.all_sell_volumes = self.all_sell_volumes + [0 for i in range(len(self.all_sell_bins) - len(self.all_sell_volumes))]
                self.all_sell_volumes[idx] += volume
                return idx

    def delete_from_buy(self, price, volume):
        max_price = self.all_buy_bins[0]
        idx = math.ceil((max_price - price) / self.price_step)
        if 0 <= idx < len(self.all_buy_bins):
            if volume < self.all_buy_volumes[idx]:
                self.all_buy_volumes[idx] -= volume
                return idx
            # find first non-zero element
            else:
                self.all_buy_volumes[idx] = 0
                while self.all_buy_volumes[idx] == 0:
                    if (idx == len(self.all_buy_volumes) - 1):
                        break
                    idx += 1
                return idx
        else:
            return -1

    def delete_from_sell(self, price, volume):
        min_price = self.all_sell_bins[0]
        idx = math.ceil((price - min_price) / self.price_step)
        if 0 <= idx < len(self.all_sell_bins):
            if volume < self.all_sell_volumes[idx]:
                self.all_sell_volumes[idx] -= volume
                return idx
            # find first non-zero element
            else:
                self.all_sell_volumes[idx] = 0
                while self.all_sell_volumes[idx] == 0:
                    if (idx == len(self.all_sell_volumes) - 1):
                        break
                    idx += 1
                return idx
        else:
            return -1

    def find_idx_sell(self, price):
        k = math.ceil((price - self.best_sell) / self.price_step)
        if k == 50:
            k = 49
        return int(k)

    def find_idx_buy(self, price):
        k = math.ceil((self.best_buy - price) / self.price_step)
        if k == 50:
            k = 49
        return int(k)

    def find_idx_spectrum_sell(self, price):
        k = math.ceil((price - self.best_sell) / self.price_step) // 5
        if k == 10:
            k = 9
        return k

    def find_idx_spectrum_buy(self, price):
        k = math.ceil((self.best_buy - price) / self.price_step) // 5
        if k == 10:
            k = 9
        return k

    def recalc_spectrum_sell(self):
        self.spectrum_sell_bins = [self.sell_bins[i] for i in range(0, 50, 5)]
        self.spectrum_sell = [sum(self.sell_volumes[i:i+5]) for i in range(0, 50, 5)]
        
    def recalc_spectrum_buy(self):
        self.spectrum_buy_bins = [self.buy_bins[i] for i in range(0, 50, 5)]
        self.spectrum_buy = [sum(self.buy_volumes[i:i+5]) for i in range(0, 50, 5)]

    def new_sell_order(self, price, volume):
        # no sell orders recorded yet
        if self.best_sell == -1:
            self.best_sell = price
            max_sell = self.best_sell + 50 * self.price_step
            self.sell_bins = [p for p in np.arange(self.best_sell, max_sell, self.price_step)]
            self.spectrum_sell_bins = [p for p in np.arange(self.best_sell, max_sell, self.price_step * 5)]
            self.sell_volumes[0] = volume
            self.spectrum_sell[0] = volume
            self.all_sell_bins = self.sell_bins.copy()
            self.all_sell_volumes = self.sell_volumes.copy()
        else:
            # sell order falls somewhere in the existing bins
            if self.best_sell <= price < self.best_sell + 50 * self.price_step:
                idx = self.find_idx_sell(price)
                if idx == 50:
                    idx = 49
                self.sell_volumes[idx] += volume
                spect_idx = self.find_idx_spectrum_sell(price)
                self.spectrum_sell[spect_idx] += volume
                _ = self.insert_to_sell(price, volume)
            else:
                # found new best, update everything
                if self.best_sell > price:
                    idx = self.insert_to_sell(price, volume)
                    self.best_sell = price
                    if idx + 50 < len(self.all_sell_bins):
                        self.sell_bins = self.all_sell_bins[idx:idx+50]
                        self.sell_volumes = self.all_sell_volumes[idx:idx+50]
                    else:
                        self.sell_bins = [p for p in np.arange(self.best_sell, self.best_sell + 50 * self.price_step, self.price_step)]
                        self.sell_volumes = self.all_sell_volumes[idx:] + [0 for i in range(50 - len(self.all_sell_volumes) + idx)]
                    self.recalc_spectrum_sell()
                # save for the later usage
                else:
                    _ = self.insert_to_sell(price, volume)
                    
    def new_buy_order(self, price, volume):
        # no buy orders recorded yet
        if self.best_buy == -1:
            self.best_buy = price
            min_buy = self.best_buy - 50 * self.price_step
            self.buy_bins = [p for p in np.arange(self.best_buy, min_buy, -self.price_step)]
            self.spectrum_buy_bins = [p for p in np.arange(self.best_buy, min_buy, -self.price_step * 5)]
            self.buy_volumes[0] = volume
            self.spectrum_buy[0] = volume
            self.all_buy_bins = self.buy_bins.copy()
            self.all_buy_volumes = self.buy_volumes.copy()
        else:
            # buy order falls somewhere in the existing bins
            if self.best_buy >= price > self.best_buy - 50 * self.price_step:
                idx = self.find_idx_buy(price)
                if idx == 50:
                    idx = 49
                self.buy_volumes[idx] += volume
                spect_idx = self.find_idx_spectrum_buy(price)
                self.spectrum_buy[spect_idx] += volume
                _ = self.insert_to_buy(price, volume)
            else:
                # found new best, update everything
                if self.best_buy < price:
                    idx = self.insert_to_buy(price, volume)
                    self.best_buy = price
                    if idx + 50 < len(self.all_buy_bins):
                        self.buy_bins = self.all_buy_bins[idx:idx+50]
                        self.buy_volumes = self.all_buy_volumes[idx:idx+50]
                    else:
                        self.buy_bins = [p for p in np.arange(self.best_buy, self.best_buy - 50 * self.price_step, -self.price_step)]
                        self.buy_volumes = self.all_buy_volumes[idx:] + [0 for i in range(50 - len(self.all_buy_volumes) + idx)]
                    self.recalc_spectrum_buy()
                # save for the later usage
                else:
                    _ = self.insert_to_buy(price, volume)
                    
    def delete_sell_order(self, price, volume):
        # does not remove current best
        if self.best_sell + 50 * self.price_step > price > self.best_sell or price == self.best_sell and volume < self.sell_volumes[0]:
            idx = self.find_idx_sell(price)
            self.sell_volumes[idx] = max(0, self.sell_volumes[idx] - volume)
            spect_idx = self.find_idx_spectrum_sell(price)
            self.spectrum_sell[spect_idx] = max(0, self.spectrum_sell[spect_idx] - volume)
        else:
            # if removes current best
            if price == self.best_sell and volume >= self.sell_volumes[0]:
                idx = self.delete_from_sell(price, volume)
                self.best_sell = self.all_sell_bins[idx]
                if idx + 50 < len(self.all_sell_bins):
                    self.sell_bins = self.all_sell_bins[idx:idx+50]
                    self.sell_volumes = self.all_sell_volumes[idx:idx+50]
                else:
                    self.sell_bins = [p for p in np.arange(self.best_sell, self.best_sell + 50 * self.price_step, self.price_step)]
                    self.sell_volumes = self.all_sell_volumes[idx:] + [0 for i in range(50 - len(self.all_sell_volumes) + idx)]
                self.recalc_spectrum_sell()
            # if does not fall in 50 steps
            elif price > self.best_sell + 50 * self.price_step:
                _ = self.delete_from_sell(price, volume)
                
    def delete_buy_order(self, price, volume):
        # does not remove current best
        if self.best_buy - 50 * self.price_step < price < self.best_buy or price == self.best_buy and volume < self.buy_volumes[0]:
            idx = self.find_idx_buy(price)
            self.buy_volumes[idx] = max(0, self.buy_volumes[idx] - volume)
            spect_idx = self.find_idx_spectrum_buy(price)
            self.spectrum_buy[spect_idx] = max(0, self.spectrum_buy[spect_idx] - volume)
        else:
            # if removes current best
            if price == self.best_buy and volume >= self.buy_volumes[0]:
                idx = self.delete_from_buy(price, volume)
                self.best_buy = self.all_buy_bins[idx]
                if idx + 50 < len(self.all_buy_bins):
                    self.buy_bins = self.all_buy_bins[idx:idx+50]
                    self.buy_volumes = self.all_buy_volumes[idx:idx+50]
                else:
                    self.buy_bins = [p for p in np.arange(self.best_buy, self.best_buy - 50 * self.price_step, -self.price_step)]
                    self.buy_volumes = self.all_buy_volumes[idx:] + [0 for i in range(50 - len(self.all_buy_volumes) + idx)]
                self.recalc_spectrum_buy()
            # if does not fall in 50 steps
            elif price > self.best_buy + 50 * self.price_step:
                _ = self.delete_from_buy(price, volume)
                
    def vwaps(self):
        # returns arrays of vwaps for(100k, 200k, 500k, 1m, 5m, 10m)
        # returns -1 if not possible to buy such a volume
        volumes = [100000, 200000, 500000, 1000000, 5000000, 10000000]
        asks = []
        bids = []
        if len(self.all_sell_bins) > 0:
            a = math.ceil((self.best_sell - self.all_sell_bins[0]) / self.price_step)
            if 0 <= a < len(self.all_sell_volumes):
                ask_cum = np.cumsum(self.all_sell_volumes[a:])
                for v in volumes:
                    p = -1
                    idx = np.argwhere(ask_cum >= v)
                    if len(idx) > 0:
                        idx = idx[0][0]
                        if ask_cum[idx] == v:
                            p = np.sum(np.multiply(self.all_sell_volumes[a:idx + 1], self.all_sell_bins[a:idx + 1])) / v
                        else:
                            p = (np.sum(np.multiply(self.all_sell_volumes[a:idx], self.all_sell_bins[a:idx])) + self.all_sell_bins[idx] * (v - ask_cum[idx-1])) / v
                    asks.append(p)
            else:
                asks = [-1] * 6
        else:
            asks = [-1] * 6
            
        if len(self.all_buy_bins) > 0:
            b = math.ceil((self.all_buy_bins[0] - self.best_buy) / self.price_step)
            if 0 <= b < len(self.all_buy_volumes):
                bid_cum = np.cumsum(self.all_buy_volumes[b:])
                for v in volumes:
                    p = -1
                    idx = np.argwhere(bid_cum >= v)
                    if len(idx) > 0:
                        idx = idx[0][0]
                        if bid_cum[idx] == v:
                            p = np.sum(np.multiply(self.all_buy_volumes[b:idx + 1], self.all_buy_bins[b:idx + 1])) / v
                        else:
                            p = (np.sum(np.multiply(self.all_buy_volumes[b:idx], self.all_buy_bins[b:idx])) + self.all_buy_bins[idx] * (v - bid_cum[idx-1])) / v
                    bids.append(p)
            else:
                bids = [-1] * 6
        else:
            bids = [-1] * 6

        # normalization
        mp = (self.best_sell + self.best_buy) / 2
        for i in range(len(asks)):
            if asks[i] != -1:
                asks[i] = np.abs(asks[i] - mp) / self.price_step
        for i in range(len(bids)):
            if bids[i] != -1:
                bids[i] = np.abs(bids[i] - mp) / self.price_step

        return asks, bids

In [4]:
class OrderBookSpectrum():

    # def __init__(self, tradelog_labels):
    def __init__(self, price_step, spectrum_base):
        self.ob_df = {}
        self.collisions = 0
        self.spectrum = Spectrum(price_step)
        self.spectrum_file_1 = open(spectrum_base + '_task_5_1.csv', "w+")
        self.spectrum_file_2 = open(spectrum_base + '_task_5_2.csv', "w+")
        self.spectrum_file_2.write('TIMESTAMP, ASK/BID, , , , , , \n')
        self.spectrum_file_5 = open(spectrum_base + '_task_5-5.csv', "w+")
        
        
    def write_5_1(self, time):
        buy_spect = np.array(self.spectrum.spectrum_buy)
        sell_spect = np.array(self.spectrum.spectrum_sell)
        buy_norm = np.flip(buy_spect / 10000000.0)
        sell_norm = sell_spect / 10000000.0
        buy = ', '.join([str(i) for i in buy_norm])
        sell = ', '.join([str(i) for i in sell_norm])
        to_write = str(time) + ', ' + sell + ', ' + buy + '\n'
        self.spectrum_file_1.write(to_write)
        
        
    def write_5_2(self, time):
        asks, bids = self.spectrum.vwaps()
        ask_write = str(time) + ', A, ' + ', '.join([str(i) for i in asks]) + '\n'
        bid_write = str(time) + ', B, ' + ', '.join([str(i) for i in bids]) + '\n'
        self.spectrum_file_2.write(ask_write)
        self.spectrum_file_2.write(bid_write)
        
    def write_5_5(self, time):
        to_write = str(time) + ', -1\n'
        if self.spectrum.best_sell != -1 and self.spectrum.best_buy != -1:
            to_write = str(time) + ', ' + str(np.abs(self.spectrum.best_sell - self.spectrum.best_buy) / self.spectrum.price_step) + '\n'
        self.spectrum_file_5.write(to_write)


    def post_order(self, orderno, volume, buysell, price, time):
        self.ob_df[orderno] = {'volume': volume, 'buysell': buysell, 'price': price, 'time': time}
        if buysell == 'B':
            self.spectrum.new_buy_order(price, volume)
        else:
            self.spectrum.new_sell_order(price, volume)


    def revoke_order(self, orderno, volume, buysell, price, time):

        if orderno in self.ob_df:
            if volume == self.ob_df[orderno]['volume']:
                self.ob_df.pop(orderno, None)
                if buysell == 'B':
                    self.spectrum.delete_buy_order(price, volume)
                else:
                    self.spectrum.delete_sell_order(price, volume)
            elif volume < self.ob_df[orderno]['volume']:
                self.ob_df[orderno]['volume'] -= volume
                if buysell == 'B':
                    self.spectrum.delete_buy_order(price, volume)
                else:
                    self.spectrum.delete_sell_order(price, volume)
            else:
                print('\nException: not possible volume for match: \n', orderno)
                self.collisions += 1
                self.delete_collision(order)
        else:
            print('\nException: orderno does not exist: \n', orderno)
            self.collisions += 1
            self.delete_collision(order)
    

    def match_order(self, orderno, volume, buysell, price, time):

        if orderno in self.ob_df:
            if volume == self.ob_df[orderno]['volume']:
                self.ob_df.pop(orderno, None)
                if buysell == 'B':
                    self.spectrum.delete_buy_order(price, volume)
                else:
                    self.spectrum.delete_sell_order(price, volume)
            elif volume < self.ob_df[orderno]['volume']:
                self.ob_df[orderno]['volume'] -= volume
                if buysell == 'B':
                    self.spectrum.delete_buy_order(price, volume)
                else:
                    self.spectrum.delete_sell_order(price, volume)
            else:
                print('\nException: not possible volume for match: \n', orderno)
                self.collisions += 1
        else:
            print('\nException: orderno does not exist: \n', orderno)
            self.collisions += 1


    def delete_collision(self, orderno):
        print('Delete collisioned orders with ORDERNO: ' + orderno)
        self.ob_df.pop(orderno, None)
        print('Current number of collisions: {}\n'.format(self.collisions))


    def new_order(self, action, orderno, volume, buysell, price, time):
        if action == 1:
            self.post_order(orderno, volume, buysell, price, time)
        elif action == 0:
            self.revoke_order(orderno, volume, buysell, price, time)
        elif action == 2:
            self.match_order(orderno, volume, buysell, price, time)
        else: 
            self.collisions += 1
        self.write_5_1(time)
        self.write_5_2(time)
        self.write_5_5(time)

In [5]:
def generate_orderbook(orderlog_path, output_path, instrument, price_step, filename):
    df = pd.read_csv(orderlog_path)
    actions = df['ACTION'].to_numpy()
    ordernos = df['ORDERNO'].to_numpy()
    volumes = df['VOLUME'].to_numpy()
    buysells = df['BUYSELL'].to_numpy()
    prices = df['PRICE'].to_numpy()
    times = df['TIME'].to_numpy()
    
    ob = OrderBookSpectrum(price_step, filename)

    for i in range(len(df)):
        ob.new_order(actions[i], ordernos[i], volumes[i], buysells[i], prices[i], times[i])

    return ob

In [ ]:
for i in tqdm(instruments[1:]):
    i_pricestep = instruments_info[i]['PRICE_STEP']
    spectrum_filename = 'task5/' + i + '_spectrum_day1'
    orderbook_filename = 'orderbooks/' + i + '_orderbook_20180301.txt'

    generate_orderbook('fx_separated/' + i + '/orderlog20180301.txt', orderbook_filename, i, i_pricestep, spectrum_filename)